<a href="https://colab.research.google.com/github/ImagingDataCommons/CloudSegmentator/blob/main/workflows/TotalSegmentator/Notebooks/downloadDicomAndConvertAndInferenceTotalSegmentatorNotebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **This Notebook can download CT data from Imaging Data Commons and convert to NIfTI with dcm2niix, perform Inference using TotalSegmentator(v1.5.6) and produces multilabel Segmentation Maps NIfTI file**

<img src="https://raw.githubusercontent.com/ImagingDataCommons/CloudSegmentator/main/workflows/TotalSegmentator/Docs/images/download_convert_inference_totalseg.png">

DICOM files are downloaded from IDC and converted to NIFTI files with dcm2niix. Whenever there are multiple NIFTI files generated for a series, such series are removed from final output. A CSV file is created with a list of such series. Inference is then performed using TotalSegmentator (v1.5.6) resulting in multilabeled Segmentation Map in NIfTI format, compressed by lz4


Please cite:

Jakob Wasserthal, Manfred Meyer, Hanns-Christian Breit, Joshy Cyriac, Shan Yang, & Martin Segeroth. (2022). TotalSegmentator: robust segmentation of 104 anatomical structures in CT images. https://doi.org/10.48550/arXiv.2208.05868

Isensee, F., Jaeger, P.F., Kohl, S.A.A. et al. nnU-Net: a self-configuring method for deep learning-based biomedical image segmentation. Nat Methods 18, 203–211 (2021). https://doi.org/10.1038/s41592-020-01008-z

Li X, Morgan PS, Ashburner J, Smith J, Rorden C. (2016) The first step for neuroimaging data analysis: DICOM to NIfTI conversion. J Neurosci Methods. 264:47-56.

Fedorov A, Longabaugh WJR, Pot D, Clunie DA, Pieper SD, Gibbs DL, Bridge C, Herrmann MD, Homeyer A, Lewis R, Aerts HJWL, Krishnaswamy D, Thiriveedhi VK, Ciausu C, Schacherer DP, Bontempi D, Pihl T, Wagner U, Farahani K, Kim E, Kikinis R. National Cancer Institute Imaging Data Commons: Toward Transparency, Reproducibility, and Scalability in Imaging Artificial Intelligence. Radiographics. 2023 Dec;43(12):e230180. doi: 10.1148/rg.230180. PMID: 37999984; PMCID: PMC10716669.

### **Ways to utilize this notebook**


*   **Colab**
*   **DockerContainer/Terra/SB-CGC**


#### **Colab**
*  This notebook was initally developed and tested on Colab, and a working version is saved on github, however reproducibility may not be guaranteed as the run time environment changes with colab updates
*  To run this notebook with Colab, Click 'Open In Colab' icon on top left
*  In 'interactive' mode, a list of seriesInstanceUIDs are chosen by default but a user may modify them to their use case
*  Run each cell to install the packages and to download the data from IDC, convert to NIfTI, perform inference using TotalSegmentator and resulting segmentation maps are saved in lz4 compressed format

#### **Docker**
* This notebook is saved by default in a way that's amenable to be used on Terra/SB-CGC platforms using Docker
* Running this notebook in a docker container ensures reproduciblity, as we lock the run environment beginning from the base docker image to pip packages in the docker image
* Docker images can be found @ https://hub.docker.com/repository/docker/imagingdatacommons/download_convert_inference_totalseg/tags
* The link to dockerfile along with git commit hash used for building the docker image can be found in one of the layers called 'LABEL'

    <img src="https://raw.githubusercontent.com/ImagingDataCommons/CloudSegmentator/main/workflows/TotalSegmentator/Docs/images/download_convert_inference_totalseg_docker.png">

* We use a python package called Papermill, that can run the notebook with out having to convert it to python script. This allows us maintain one copy of code instead of two.
* A sample papermill command is
    <pre>
    papermill downloadDicomAndConvertAndInferenceTotalSegmentatorNotebook.ipynb outputdownloadDicomAndConvertAndInferenceTotalSegmentatorNotebook.ipynb -y SeriesInstanceUIDs yamlListOfSeriesInstanceUIDs
    </pre>
    * refer to https://papermill.readthedocs.io/en/latest/usage-execute.html#note-about-using-yaml on how to pass the yaml list

    Instead of passing the yaml list as a string, a yaml file can also be passed, as we do for SB-CGC
    [an example can be found here](!https://raw.githubusercontent.com/ImagingDataCommons/CloudSegmentator/main/workflows/TotalSegmentator/Docs/sampleManifests/batch_1.yaml)
    <pre>
    papermill downloadDicomAndConvertAndInferenceTotalSegmentatorNotebook.ipynb outputdownloadDicomAndConvertAndInferenceTotalSegmentatorNotebook.ipynb -f path_to_yamlListOfSeriesInstanceUIDs.yaml
    </pre>

### **Installing Packages**

In [ ]:
%%capture
import sys
if 'google.colab' in sys.modules:
    !sudo apt-get update && apt-get install -y --no-install-recommends \
        build-essential\
        dcm2niix\
        ffmpeg\
        lz4\
        pigz\
        #plastimatch=1.8.0+dfsg.1-2build1\
        python3-dev\
        python3-pip\
        unzip\
        wget\
        xvfb\
        zip\
        && rm -rf /var/lib/apt/lists/*

In [ ]:
%%capture
if 'google.colab' in sys.modules:
    !sudo pip install --no-cache-dir\
        idc_index==0.2.8\
        ipykernel==6.22.0\
        ipython==8.12.0\
        ipywidgets==8.0.6\
        jupyter==1.0.0\
        nvidia-ml-py3==7.352.0\
        papermill==2.4.0 \
        requests==2.27.1\
        TotalSegmentator==1.5.6

In [ ]:
%%capture
if 'google.colab' in sys.modules:
    #install dcm2niix
    !wget "https://github.com/rordenlab/dcm2niix/releases/download/v1.0.20230411/dcm2niix_lnx.zip" \
        && unzip "dcm2niix_lnx.zip" \
        && rm "dcm2niix_lnx.zip" \
        && mv dcm2niix /usr/local/bin/dcm2niix

In [ ]:
%%capture
if 'google.colab' in sys.modules:
    #install s5cmd
    !wget "https://github.com/peak/s5cmd/releases/download/v2.2.2/s5cmd_2.2.2_Linux-64bit.tar.gz"
    !tar -xvzf "s5cmd_2.2.2_Linux-64bit.tar.gz"
    !rm "s5cmd_2.2.2_Linux-64bit.tar.gz"
    !mv s5cmd /usr/local/bin/s5cmd

### **Importing Packages**

In [ ]:
from concurrent.futures import ThreadPoolExecutor
from datetime import datetime
from idc_index import index
from pathlib import Path
from time import sleep
import glob
import matplotlib.pyplot as plt
import nvidia_smi
import os
import pandas as pd
import psutil
import shutil
import subprocess
import sys
import time

### **Current Environment**

In [ ]:
curr_dir   = Path().absolute()

os.environ['TZ'] = 'US/Eastern'
time.tzset()
current_time = time.strftime('%a %b %d %H:%M:%S %Y', time.localtime())
print(current_time)
print("\nCurrent directory :{}".format( curr_dir))
print("Python version    :", sys.version.split('\n')[0])

### **Initialize IDC Client**

We use idc-client pypi package to handle downloading data from IDC. 
In this notebook, we are using version 0.2.8 which contains the index from idc version 17

Learn more about idc-index at ttps://github.com/ImagingDataCommons/idc-index

In [ ]:
idc_client=index.IDCClient()

### **Local testing**
By default, in interactive mode, a list of seriesInstanceUIDs are chosen  here. However, you can modify them to your usecase.

Below cell is also tagged as `parameters`, so that when running this notebook in non interactive mode on Terra or Seven Bridges Genomics- Cancer Genomics Cloud platforms, papermill will inject a cell to pass the yaml list of SeriesInstanceUIDs

In [ ]:
if 'google.colab' in sys.modules:
  SeriesInstanceUIDs=['1.3.6.1.4.1.14519.5.2.1.7009.9004.100143549999116733615345241533',
                      '1.3.6.1.4.1.14519.5.2.1.7009.9004.100148350742920339334061834697',
                      '1.3.6.1.4.1.14519.5.2.1.7009.9004.100241427395754063917290539621',
                      '1.3.6.1.4.1.14519.5.2.1.7009.9004.100266844261017577841946689119',
                      '1.3.6.1.4.1.14519.5.2.1.7009.9004.100554983367710060268444607770',
                      '1.3.6.1.4.1.14519.5.2.1.7009.9004.100609092457306482866656779396',
                      '1.3.6.1.4.1.14519.5.2.1.7009.9004.100766949446324115207263771273',
                      '1.3.6.1.4.1.14519.5.2.1.7009.9004.100807683971079396484581858934',
                      '1.3.6.1.4.1.14519.5.2.1.7009.9004.100898243941555041874482639283',
                      '1.3.6.1.4.1.14519.5.2.1.7009.9004.100962349324934756610763981593',
                      '1.3.6.1.4.1.14519.5.2.1.7009.9004.100993923831797845405183790593',
                      '1.3.6.1.4.1.14519.5.2.1.7009.9004.101126369092366339550409924127'
                      ]



### **Defining Functions**

In [ ]:
#create directory for TotalSegmentator Output files
try:
  shutil.rmtree('dcm2niix')
  shutil.rmtree('Inference')

except OSError:
  pass
os.mkdir('dcm2niix')
os.mkdir('Inference')

In [ ]:
def download_dicom_data(series_id: str) -> None:
    """
    Downloads raw DICOM data

    Args:
    series_id: The DICOM Tag SeriesInstanceUID of the DICOM series to be converted.
    """
    download_directory = f"idc_data/{series_id}"
    # Attempt to remove the directory for the series if it exists
    try:
        shutil.rmtree(download_directory)
    except OSError:
        pass
    print(f'\n Downloading DICOM files from IDC Storage Buckets \n')
    idc_client.download_dicom_series(seriesInstanceUID= series_id, downloadDir=download_directory, quiet=False)



In [ ]:
def is_series_CT(series_id: str) -> bool:
    """
    Gets the image modality for the corresponding seriesInstanceUID from idc-index
    Refer to this query for additional columns available in idc-index!

    https://github.com/ImagingDataCommons/idc-index/blob/main/queries/idc_index.sql

    Args:
    series_id: The DICOM Tag SeriesInstanceUID of the DICOM series to be processed.
    """

    query = f"""
    SELECT
    Modality
    FROM index
    WHERE SeriesInstanceUID = '{series_id}'
    """

    try:
        modality_df = idc_client.sql_query(query)
        if not modality_df.empty:
            modality = modality_df['Modality'][0]
            if modality=='CT':
              return True
            else:
              log_modality_errors(series_id)
              return False
        else:
            log_modality_errors(series_id)
            return False
    except Exception as e:
        print(f"An error occurred: {e}")
        log_modality_errors(series_id)
        return False


In [ ]:
def log_modality_errors(series_id: str) -> None:
    """
    Logs an error when the modality is not CT for a given series.

    Args:
        series_id: The ID of the series.
    """
    # Open the log file in append mode
    with open("modality_error_file.txt", "a") as f:
        # Write the error message to the file
        f.write(f"Error: Modality is not CT for series {series_id}\n")


In [ ]:
def convert_dicom_to_nifti(series_id: str) -> None:
    """
    Converts a DICOM series to a NIfTI file.

    Args:
      series_id: The DICOM Tag SeriesInstanceUID of the DICOM series to be converted.
    """

    # Attempt to remove the directory for the series if it exists
    try:
        shutil.rmtree(f"dcm2niix/{series_id}")
    except OSError:
        pass

    # Create a new directory for the series
    os.mkdir(f"dcm2niix/{series_id}")

    print("\n Converting DICOM files to NIfTI \n")

    # Run the appropriate converter command and capture the output

    result = subprocess.run(
        f"dcm2niix -z y -f %j_%p_%t_%s -b n -m y -o dcm2niix/{series_id} idc_data/{series_id}",
        shell=True,
        capture_output=True,
        text=True,
    )
    print(result.stdout)
    print("\n Conversion successful")
    #cleaning up the dicom files directory as they are no longer needed
    shutil.rmtree(f'idc_data/{series_id}')


In [ ]:
def check_dicom_conversion_errors(series_id_folder_path):
    """
    This function checks if the conversion from DICOM to NIfTI format was successful.
    It does this by checking the number of files in the specified folder.
    The conversion is considered successful if there is exactly one file in the folder.

    Args:
    series_id_folder_path (str): The path of the folder containing the converted NIfTI files.

    Returns:
    bool: True if there was an error in the conversion
    (i.e., no files or more than one file in the folder), False otherwise.
    """

    # Get a list of all files in the specified folder
    nifti_files = os.listdir(series_id_folder_path)

    # Check if the folder is empty
    if len(nifti_files) == 0:
        # If the folder is empty, log an error message in 'error_file.txt'
        # This indicates that no file was created during the conversion, which means an error occurred
        with open('error_file.txt', 'a') as f:
            f.write(f"Error: No files in {series_id_folder_path}\n")
        # Return True to indicate an error
        return True

    # Check if the folder contains more than one file
    elif len(nifti_files) > 1:
        # If the folder contains more than one file, log an error message in 'error_file.txt'
        # This indicates that more than one file was created during the conversion, which should not happen and thus means an error occurred
        with open('error_file.txt', 'a') as f:
            f.write(f"Error: More than one file in {series_id_folder_path}\n")
        # Return True to indicate an error
        return True

    # If there is exactly one file in the folder, no error occurred during the conversion
    else:
        return False


In [ ]:
def check_total_segmentator_errors(series_id: str):
  """
  This function checks if the output files from TotalSegmentator exist.

  Args:
  series_id (str): The DICOM Tag SeriesInstanceUID of the DICOM series to be checked.

  Returns:
  bool: True if any of the output files do not exist, False otherwise.
  """

  # Define the output files from TotalSegmentator
  output_files = [f"Inference/{series_id}/segmentations.nii"]

  # Check if all output files exist
  if not all(os.path.exists(file) for file in output_files):
      # If any of the output files do not exist, log an error
      with open('totalsegmentator_errors.txt', 'a') as f:
          f.write(f"Error: TotalSegmentator failed for series {series_id}\n")
      return True

  return False


In [ ]:
def inferenceTotalSegmentator(series_id: str,runtime_stats: pd.DataFrame) -> pd.DataFrame:
    """
    This function calls download_dicom_data and then performs inference using TotalSegmentator on a given series.

    Args:
    series_id (str): The DICOM Tag SeriesInstanceUID of the DICOM series to be processed.
    runtime_stats: DataFrame to store runtime statistics.
    """
    print("Processing series: " + series_id)

    if is_series_CT(series_id):

        start_time = time.time()
        download_dicom_data(series_id)
        dicom_download_time = time.time() - start_time

        start_time = time.time()
        convert_dicom_to_nifti(series_id)
        dicom_conversion_time = time.time() - start_time

        # Remove existing directories and files if they exist
        shutil.rmtree(f"Inference/{series_id}", ignore_errors=True)
        shutil.rmtree(f"metadata/{series_id}", ignore_errors=True)
        try:
            os.remove("segmentations.nii.gz")
        except OSError:
            pass

        # Create a new directory for the series
        os.makedirs(f"Inference/{series_id}", exist_ok=True)

        series_id_folder_path = os.path.join("dcm2niix", series_id)

        if not check_dicom_conversion_errors(series_id_folder_path):
            # Get the list of files in series_id_path
            nifti_files = os.listdir(series_id_folder_path)
            # Get the first (and only) file in the list
            nifti_filename = nifti_files[0]
            # Get the full path of the file
            nifti_filename_path = os.path.join(series_id_folder_path, nifti_filename)

            start_time = time.time()
            result = subprocess.run(
                [
                    "TotalSegmentator",
                    "-i",
                    nifti_filename_path,
                    "-o",
                    "segmentations",
                    "--ml",
                ],
                stdout=subprocess.PIPE,
                stderr=subprocess.PIPE,
                universal_newlines=True,
            )
            print(result.stdout)
            total_segmentator_time = time.time() - start_time

            try:
                subprocess.run(
                    ["mv", "segmentations.nii", f"Inference/{series_id}/"], check=True
                )
                print("Files moved successfully using the first command")
            except subprocess.CalledProcessError:
                try:
                    subprocess.run(
                        [
                            "mv",
                            "segmentations/segmentations.nii",
                            f"Inference/{series_id}/",
                        ],
                        check=True,
                    )
                    print("Files moved successfully using the second command")
                except subprocess.CalledProcessError:
                    print("Error: Failed to move files using both commands")

            check_total_segmentator_errors(series_id)

            shutil.move(
                f"Inference/{series_id}/segmentations.nii",
                f"Inference/{series_id}/{series_id}.nii",
            )

            start_time = time.time()
            subprocess.run(
                [
                    "lz4",
                    "--rm",
                    f"Inference/{series_id}/{series_id}.nii",
                    f"Inference/{series_id}/{series_id}.nii.lz4",
                ],
                check=True,
            )

            archiving_time = time.time() - start_time

        else:
            total_segmentator_time = 0
            archiving_time = 0

    else:
        dicom_download_time = 0
        dicom_conversion_time = 0
        total_segmentator_time = 0
        archiving_time = 0

    log = pd.DataFrame({"SeriesInstanceUID": [series_id]})
    log["dicom_download_time"] = dicom_download_time
    log["dicom_conversion_time"] = dicom_conversion_time
    log["total_segmentator_time"] = total_segmentator_time
    log["archiving_time"] = archiving_time

    shutil.rmtree(f"dcm2niix/{series_id}", ignore_errors=True)

    runtime_stats = pd.concat([runtime_stats, log], ignore_index=True, axis=0)

    return runtime_stats


In [ ]:
class MemoryMonitor:
    def __init__(self):
        self.keep_measuring = True
        self.working_disk_path = self.get_working_disk_path()

    def get_working_disk_path(self):
        partitions = psutil.disk_partitions()
        for partition in partitions:
            if partition.mountpoint == '/':
                return '/'
            elif '/cromwell_root' in partition.mountpoint:
                return '/cromwell_root'
        return '/'  # Default to root directory if no specific path is found
    def measure_usage(self):
        cpu_usage = []
        ram_usage_mb=[]
        gpu_usage_mb=[]
        disk_usage_all=[]
        time_stamps = []
        start_time = time.time()
        while self.keep_measuring:
            cpu = psutil.cpu_percent()
            ram = psutil.virtual_memory()
            disk_usage = psutil.disk_usage(self.working_disk_path)
            disk_used = disk_usage.used / 1024 / 1024 / 1024
            disk_total = disk_usage.total / 1024 / 1024 / 1024
            ram_total_mb = psutil.virtual_memory().total / 1024 / 1024
            ram_mb = (ram.total - ram.available) / 1024 / 1024
            try:
                nvidia_smi.nvmlInit()
                handle = nvidia_smi.nvmlDeviceGetHandleByIndex(0)
                info = nvidia_smi.nvmlDeviceGetMemoryInfo(handle)
                gpu_type = nvidia_smi.nvmlDeviceGetName(handle)
                gpu_total_mb = info.total/1024/1024
                gpu_mb = info.used/1024/1024
                nvidia_smi.nvmlShutdown()
            except:
                gpu_type = ''
                gpu_total_mb = 0
                gpu_mb = 0
                
            cpu_usage.append(cpu)
            ram_usage_mb.append(ram_mb)
            disk_usage_all.append(disk_used)
            gpu_usage_mb.append(gpu_mb)
            time_stamps.append(time.time()- start_time)
            sleep(1)

        return cpu_usage, ram_usage_mb, time_stamps, ram_total_mb, gpu_usage_mb, gpu_total_mb, gpu_type, disk_usage_all, disk_total

###Total Segmentator

In [ ]:
if __name__ == "__main__":
    # Initialize a DataFrame to store runtime statistics
    runtime_stats = pd.DataFrame(columns=['SeriesInstanceUID','total_segmentator_time','dicom_download_time','dicom_conversion_time',
                                          'archiving_time', 'cpu_usage','ram_usage_mb', 'ram_total_mb',
                                          'gpu_usage_mb', 'gpu_total_mb', 'gpu_type', 'disk_usage_all', 'disk_total'
                                          ])

    for series_id in SeriesInstanceUIDs:
        with ThreadPoolExecutor() as executor:
            monitor = MemoryMonitor()
            mem_thread = executor.submit(monitor.measure_usage)
            try:
                proc_thread = executor.submit(inferenceTotalSegmentator, series_id, runtime_stats)
                runtime_stats = proc_thread.result()
            finally:
                monitor.keep_measuring = False
                cpu_usage, ram_usage_mb, time_stamps, ram_total_mb, gpu_usage_mb, gpu_total_mb, gpu_type, disk_usage_all, disk_total= mem_thread.result()

                cpu_idx = runtime_stats.index[runtime_stats['SeriesInstanceUID'] == series_id][0]
                runtime_stats.iloc[cpu_idx, runtime_stats.columns.get_loc('cpu_usage')] = [[cpu_usage]]

                ram_usage_mb_idx = runtime_stats.index[runtime_stats['SeriesInstanceUID'] == series_id][0]
                runtime_stats.iloc[ram_usage_mb_idx, runtime_stats.columns.get_loc('ram_usage_mb')] = [[ram_usage_mb]]

                ram_total_mb_idx = runtime_stats.index[runtime_stats['SeriesInstanceUID'] == series_id][0]
                runtime_stats.iloc[ram_total_mb_idx, runtime_stats.columns.get_loc('ram_total_mb')] = [[ram_total_mb]]

                gpu_total_mb_idx = runtime_stats.index[runtime_stats['SeriesInstanceUID'] == series_id][0]
                runtime_stats.iloc[gpu_total_mb_idx, runtime_stats.columns.get_loc('gpu_total_mb')] = [[gpu_total_mb]]

                gpu_usage_mb_idx = runtime_stats.index[runtime_stats['SeriesInstanceUID'] == series_id][0]
                runtime_stats.iloc[gpu_usage_mb_idx, runtime_stats.columns.get_loc('gpu_usage_mb')] = [[gpu_usage_mb]]

                disk_usage_gb_idx = runtime_stats.index[runtime_stats['SeriesInstanceUID'] == series_id][0]
                runtime_stats.iloc[disk_usage_gb_idx, runtime_stats.columns.get_loc('disk_usage_all')] = [[disk_usage_all]]

                runtime_stats['gpu_type']=gpu_type
                runtime_stats['disk_total']=disk_total

                fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2,2, figsize=(8, 6))

                ax1.plot(time_stamps, cpu_usage)
                ax1.set_ylim(0, 100)
                ax1.set_xlabel('Time (s)')
                ax1.set_ylabel('CPU usage (%)')

                ax2.plot(time_stamps, ram_usage_mb)
                ax2.set_ylim(0, ram_total_mb)
                ax2.set_xlabel('Time (s)')
                ax2.set_ylabel('Memory usage (MB)')

                ax3.plot(time_stamps, gpu_usage_mb)
                ax3.set_ylim(0, gpu_total_mb)
                ax3.set_xlabel('Time (s)')
                ax3.set_ylabel('GPU Memory usage (MB)')

                ax4.plot(time_stamps, disk_usage_all)
                ax4.set_ylim(0, disk_total)
                ax4.set_xlabel('Time (s)')
                ax4.set_ylabel('Disk usage (GB)')
                plt.show()

### **Compressing Output Files**

In [ ]:
start_time = time.time()
try:
  os.remove('inferenceNiftiFiles.tar.lz4')
  os.remove('metadata.tar.lz4')
except OSError:
  pass
!tar cvf - -C {curr_dir} Inference | lz4 > inferenceNiftiFiles.tar.lz4

output_file_archiving_time = time.time() - start_time


### **Utilization Metrics**

In [ ]:
runtime_stats.to_csv('runtime.csv')
runtime_stats['output_file_archiving_time']=output_file_archiving_time
try:
  os.remove('inferenceUsageMetrics.lz4')
except OSError:
  pass
!lz4 {curr_dir}/runtime.csv inferenceUsageMetrics.lz4
runtime_stats